In [1]:
from timeit import default_timer as timer
import itertools
import os
import sys
import uuid
from glob import glob
import json
import tweepy
import numpy as np
import pandas as pd
import multiprocessing as mp
import psutil
import socket
from functools import partial
import time

# Params

In [2]:
cutoff = 100
print('Save Data After Downloading',cutoff,'Timelines')

Save Data After Downloading 100 Timelines


In [3]:
def get_env_var(varname,default):
    
    if os.environ.get(varname) != None:
        var = int(os.environ.get(varname))
        print(varname,':', var)
    else:
        var = default
        print(varname,':', var,'(Default)')
    return var

# Choose Number of Nodes To Distribute Credentials: e.g. jobarray=0-4, cpu_per_task=20, credentials = 90 (<100)
SLURM_JOB_ID            = get_env_var('SLURM_JOB_ID',0)
SLURM_ARRAY_TASK_ID     = get_env_var('SLURM_ARRAY_TASK_ID',0)
SLURM_ARRAY_TASK_COUNT  = get_env_var('SLURM_ARRAY_TASK_COUNT',1)
SLURM_JOB_CPUS_PER_NODE = get_env_var('SLURM_JOB_CPUS_PER_NODE',mp.cpu_count())

SLURM_JOB_ID : 0 (Default)
SLURM_ARRAY_TASK_ID : 0 (Default)
SLURM_ARRAY_TASK_COUNT : 1 (Default)
SLURM_JOB_CPUS_PER_NODE : 16 (Default)


In [4]:
path_to_data='../data'
path_to_users = os.path.join(path_to_data + '/users')
path_to_keys = os.path.join('../keys')
path_to_timelines = os.path.join(path_to_data,'timelines','API')
os.makedirs(path_to_timelines, exist_ok=True)
print(path_to_users)
print(path_to_keys)
print(path_to_timelines)


../data/users
../keys
../data/timelines/API


# Credentials

In [5]:
def get_key_files(SLURM_ARRAY_TASK_ID,SLURM_ARRAY_TASK_COUNT,SLURM_JOB_CPUS_PER_NODE):

    # Randomize set of key files using constant seed
    np.random.seed(0)
    all_key_files = np.random.permutation(glob(os.path.join(path_to_keys,'key*')))
    auth_file = np.random.permutation(glob(os.path.join(path_to_keys,'auth*')))
    
    # Split file list by node
    key_files = np.array_split(all_key_files,SLURM_ARRAY_TASK_COUNT)[SLURM_ARRAY_TASK_ID]
    
    # Check that node has more CPU than key file 
    if len(key_files) <= SLURM_JOB_CPUS_PER_NODE:
        print('# Credentials Allocated To Node:', len(key_files)) 
    else:
        print('Check environment variables:')
        print('# Credentials (',len(key_files),') > # CPU (', SLURM_JOB_CPUS_PER_NODE,')')
        print('Only keeping', SLURM_JOB_CPUS_PER_NODE, 'credentials')
        key_files = key_files[:SLURM_JOB_CPUS_PER_NODE]
        
    return key_files, auth_file

key_files, auth_file = get_key_files(SLURM_ARRAY_TASK_ID,SLURM_ARRAY_TASK_COUNT,SLURM_JOB_CPUS_PER_NODE)
print('\n'.join(key_files))
print('\n'.join(auth_file))

# Credentials Allocated To Node: 10
../keys/key_jihanne.json
../keys/key_othmane.json
../keys/key_naila.json
../keys/key_sam.json
../keys/key_cyril.json
../keys/key_noemie.json
../keys/key_youssr.json
../keys/key_marc.json
../keys/key_clemence.json
../keys/key_naila2.json
../keys/auth_naila.json


In [6]:
def get_auth(key_file):
    
    # Import Auth keys
    for auth_file in glob(os.path.join(path_to_keys,'auth*')) :
        with open (auth_file) as f:
            auth_key = json.load(f)
    
    # Import token pairs
    with open(key_file) as f:
        key = json.load(f)

    # OAuth process, using the keys and tokens
    auth = tweepy.OAuthHandler(auth_key['consumer_key'], auth_key['consumer_secret'])
    auth.set_access_token(key['access_token'], key['access_token_secret'])

    # Creation of the actual interface, using authentication
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    
    try:
        api.verify_credentials()
        print(key_file,": Authentication checked")
    except:
        print(key_file,": error during authentication")
        sys.exit('Exit')
    
    return auth, api

# Users list

In [7]:
print('Import Users in IDF and Paris')
users_idf = pd.read_json(os.path.join(path_to_users, 'users_idf.json'), lines=True)[0].tolist()
users_paris = pd.read_json(os.path.join(path_to_users, 'users_paris.json'), lines=True)[0].tolist()

Import Users in IDF and Paris


In [34]:
print("Users for which we don't have the timeline")
with open(os.path.join(path_to_timelines,'IDF','success','downloaded_ids.txt'), 'r', encoding='utf-8') as file:
    users_idf_downloaded = file.read().splitlines()
users_idf_to_download = [item for item in users_idf if item not in users_idf_downloaded]

Users for which we don't have the timeline


# Get timeline

In [8]:
def get_timeline(user_id,api):
    
    timeline = []
    error = None
    
    # Collect All Statuses in Timeline
    try:
        cursor = tweepy.Cursor(
        api.user_timeline, 
        user_id=user_id, 
        count=3200,
        tweet_mode="extended", 
        include_rts=True, 
        sleep_on_rate_limit=False).items()
        
        for status in cursor:
            timeline.append(status._json)
     
    except tweepy.error.TweepError as e:
        error = str(e)
    timeline = pd.DataFrame(timeline, columns = ['id_str','user','full_text','created_at','lang'])
    timeline['user_id'] = timeline['user'].apply(lambda x: x.get('id_str'))
    timeline['user_name'] = timeline['user'].apply(lambda x: x.get('name'))
    return timeline[['id_str','user_id','user_name','full_text','created_at','lang']], error

# timeline = get_user_timeline('12',get_auth(key_file))

In [38]:
cutoff=100

def download_timelines(index_key,users_list):

    # Create Access For Block of Users
    api = get_auth(key_files[index_key])[1]
    
    # Select Block of Users
    users_block = np.array_split(users_list,len(key_files))[index_key]
    
    # Initialize Output File ID
    output_id = str(uuid.uuid4())
    
    # Initialize DataFrame
    timelines = pd.DataFrame()
    
    # Initialize Downloaded User List
    downloaded_ids = []
    
    for user_id in users_block:
        #print(str(user_id))
        # Try Downloading Timeline
        timeline, error = get_timeline(user_id,api)
        
        if error!=None:
            print(user_id,index_key,error)
            continue
        # Append
        timelines = pd.concat([timelines, timeline],sort=False)
        downloaded_ids.append(user_id)

        #print('appended')

        # Save after <cutoff> timelines or when reaching last user
        if len(downloaded_ids) == cutoff or user_id == users_block[-1]:
            print('save')

            filename = \
            'timelines-'+\
            str(SLURM_JOB_ID)+'-'+\
            str(SLURM_ARRAY_TASK_ID)+'-'+\
            str(index_key)+'-'+\
            str(len(downloaded_ids))+'-'+\
            output_id+'.json'
            
            print('Process', index_key, 'saving', len(downloaded_ids), 'timelines with output file:', 
            os.path.join(path_to_timelines,'IDF',filename))
            
            # Save as list of dict discarding index
            print('save as list')
            timelines.to_json(
            os.path.join(path_to_timelines,'IDF',filename),
            orient='records',
            #force_ascii=False,
            date_format=None,
            double_precision=15)
            
            # Save User Id and File In Which Its Timeline Was Saved
            print('save user_id')
            with open(os.path.join(path_to_timelines,'IDF','success','downloaded_ids.txt'), 'a', encoding='utf-8') as file:
                for downloaded_id in downloaded_ids:
                    file.write(str(downloaded_id)+'\n')
            print('user_id_saved')
            
            # Reset Output File ID, Data, and Downloaded Users
            print('reset')
            del timelines, downloaded_ids
            output_id = str(uuid.uuid4())
            timelines = pd.DataFrame()
            downloaded_ids = []
    return 0

In [ ]:
print('Extract Timelines...\n')
with mp.Pool() as pool:
    #pool.map(partial(download_timelines, users_list=users_idf), range(len(key_files)))
    pool.map(partial(download_timelines, users_list=users_idf_to_download), range(len(key_files)))

Extract Timelines...

../keys/key_naila.json : Authentication checked
../keys/key_cyril.json : Authentication checked
../keys/key_jihanne.json : Authentication checked
../keys/key_othmane.json : Authentication checked
../keys/key_sam.json : Authentication checked
../keys/key_noemie.json : Authentication checked
../keys/key_youssr.json : Authentication checked
../keys/key_clemence.json : Authentication checked
../keys/key_marc.json : Authentication checked
../keys/key_naila2.json : Authentication checked
464903859 4 Twitter error response: status code = 401
1057794546 0 Twitter error response: status code = 401
2223133986 6 Twitter error response: status code = 401
143728976 3 Twitter error response: status code = 401
1239295644336115713 0 Twitter error response: status code = 401
739568210 7 Twitter error response: status code = 401


Rate limit reached. Sleeping for: 520
Rate limit reached. Sleeping for: 518


1251619861656109058 6 Twitter error response: status code = 401


Rate limit reached. Sleeping for: 497
Rate limit reached. Sleeping for: 496
Rate limit reached. Sleeping for: 482
Rate limit reached. Sleeping for: 481
Rate limit reached. Sleeping for: 478
Rate limit reached. Sleeping for: 473
Rate limit reached. Sleeping for: 469
Rate limit reached. Sleeping for: 464


28349211 2 Twitter error response: status code = 401
988407190057897984 8 Twitter error response: status code = 503
1078627279696265216 8 Twitter error response: status code = 401
1187734345580199937 0 Twitter error response: status code = 401
340422281 7 Twitter error response: status code = 401
1150518223273697280 9 Twitter error response: status code = 401
1248643806922244100 2 Twitter error response: status code = 401
1119976115672633344 8 Twitter error response: status code = 401
154603854 1 Twitter error response: status code = 401
save
Process 5 saving 100 timelines with output file: ../data/timelines/API/IDF/timelines-0-0-5-100-fc38e4fc-f2d7-4413-afb5-11f307ce04e6.json
save as list
save user_id
user_id_saved
reset
107728519 6 Twitter error response: status code = 401
1247616994792308741 5 Twitter error response: status code = 401
save
Process 6 saving 100 timelines with output file: ../data/timelines/API/IDF/timelines-0-0-6-100-0a0f76d0-95d8-4358-881f-4ba52ec9533e.json
save as 

...

976418336132214785 0 Twitter error response: status code = 401


Rate limit reached. Sleeping for: 404


save
Process 3 saving 100 timelines with output file: ../data/timelines/API/IDF/timelines-0-0-3-100-b9442484-2d8e-4cec-b69f-7fb55367510a.json
save as list
save user_id
user_id_saved
reset


Rate limit reached. Sleeping for: 384
Rate limit reached. Sleeping for: 384
Rate limit reached. Sleeping for: 380
Rate limit reached. Sleeping for: 379
Rate limit reached. Sleeping for: 358
Rate limit reached. Sleeping for: 364
Rate limit reached. Sleeping for: 376
Rate limit reached. Sleeping for: 350


1213416215713968128 7 Twitter error response: status code = 401
1209597298726060032 7 Twitter error response: status code = 401
582976740 9 Twitter error response: status code = 401
618340609 4 Twitter error response: status code = 401
save
Process 8 saving 100 timelines with output file: ../data/timelines/API/IDF/timelines-0-0-8-100-ea3daf59-b613-4621-9fbb-f812f0018ebf.json
save as list
save user_id
user_id_saved
reset
1244579848594612232 9 Twitter error response: status code = 401
1250577438851190784 4 Twitter error response: status code = 401
save
Process 9 saving 100 timelines with output file: ../data/timelines/API/IDF/timelines-0-0-9-100-f596100c-c51b-480d-834c-e911085bc242.json
save as list
save user_id
user_id_saved
reset
1008171770132787200 4 Twitter error response: status code = 401
1126220377435398144 2 Twitter error response: status code = 401
save
Process 5 saving 100 timelines with output file: ../data/timelines/API/IDF/timelines-0-0-5-100-2aea6d29-5a3d-4d8c-98cd-408853fb